In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) #Applies the Grayscale transform (image to black and white)
    blur = cv2.GaussianBlur(gray,( 5 , 5 ), 0)     #Applies a Gaussian Noise kernel (Removes Noice)
    canny = cv2.Canny(blur , 50 ,150)   # Applies the Canny transform     
    return canny

In [ ]:
def region_of_interest(img):
    """
    Applies an image mask.    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    height = img.shape[0]
    width = img.shape[1]
    #vertices for our traingle of interest
    polygons = np.array([
        [ (0, height), (width / 2, height / 3), (width, height) ]
    ], dtype=np.int32)                                                
    
    mask = np.zeros_like(img)   #black mask image with same shape as the image
    cv2.fillPoly(mask, polygons , 255 )   #fill the mask image with the traingle
    masked_img = cv2.bitwise_and(img, mask)  #Cropping the area of interest wusing AND operator between the image and the mask
    return masked_img

In [ ]:
def display_lines(img , lines):
    line_img = np.zeros_like(img)
    if lines is not None:
        for x1,y1, x2, y2 in lines:
            cv2.line(line_img , (x1,y1), (x2,y2) , (255, 0, 0), 10 )
    return line_img

In [ ]:
def avg_slope_intercept(img , lines ):
    left_fit = []
    right_fit = []
    for line in lines:
        x1,y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2), (y1,y2) ,1 )
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
    
    if left_fit:
        left_fit_avg = np.average(left_fit, axis=0)
        left_line = make_coordinates(img , left_fit_avg)
    else:
        left_line = [0,0,0,0]
        
    if right_fit:
        right_fit_avg = np.average(right_fit, axis=0)
        right_line = make_coordinates(img , right_fit_avg)
    else:
        right_line = [0,0,0,0]
        
    #print("Right:",right_line)
    #print("Left:",left_line)

    #left_line = make_coordinates(img , left_fit_avg)
    #right_line = make_coordinates(img , right_fit_avg)
    
    
    
    return np.array([left_line, right_line])

In [ ]:
def make_coordinates(image , line_param):
    try:
        slope, intercept = line_param
    except TypeError:
        slope, intercept = 0,0
        
    y1 = image.shape[0]
    y2 = int(y1 * (3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)

    return np.array([x1, y1, x2, y2])

In [ ]:
def get_lines_image(image):
    
    #image = cv2.imread(file)  # if image path is passed use this
    #image = file               # if frames are passed 
    #print(image.shape)
    
    height = image.shape[0]
    width = image.shape[1]
    
    canny_image = canny(image)
    
    cropped_image =  region_of_interest(canny_image)
    
    lines = cv2.HoughLinesP( cropped_image , 2 , np.pi/180 , 100 , np.array([]) , minLineLength = 40 , maxLineGap = 5)
    
    avg_lines = avg_slope_intercept(image , lines)
    
    line_image = display_lines(image, avg_lines)
    
    merged_image = cv2.addWeighted(image, 0.8 , line_image ,1 ,1)
    
    return merged_image    #return line_image for model labels

In [ ]:
#For video
def video_lines(file):
    cap = cv2.VideoCapture(file)
    while(cap.isOpened()):
        _, frame = cap.read()
        images = get_lines_image(frame)
        cv2.imshow( 'result' , images ) # Change to line_image for Labels
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return

In [ ]:
file = "test2.mp4"
video_lines(file)

OverflowError: Python int too large to convert to C long

In [ ]:
file = "690 img 2.jpg" 

In [ ]:
cv2.imshow( 'result' , get_lines_image(file) )
cv2.waitKey(0) # Use this for image

In [ ]:
file